In [1]:
#todo
#scrivere se il sito è offline (gestire response code diversi da 200)
#qualcosa da aggiustare sull'inserimento della lingua


In [2]:
import pandas as pd
from stop_words import get_stop_words
import re
from selenium import webdriver
import time
from langdetect import detect, detect_langs

In [3]:
#driver = webdriver.Chrome("webdriver/chromedriver")

In [4]:
#stopwords base importate

stop_words = get_stop_words('it')
stop_words += get_stop_words('en')
stop_words += get_stop_words('pt')
stop_words += get_stop_words('es')
stop_words += get_stop_words('de')
stop_words += get_stop_words('fr')

In [5]:
#stopwords personali aggiunte da me

stop_words += ["priority","target","events","eventname","flavors","callback","document","stato","legacy","window","visualizza"]
stop_words += ["module","article","cette","encore","faire","d’une"]
stop_words += ["c'est","della","delle"]
stop_words += ["settembre",	"positivi",	"giorno",	"giorni",	"agosto",	"settimana",	"progetto",	"weeks",	"evidenza",	"siniscola",	"della",	"lucia",	"santa"]

In [6]:
df = pd.read_excel("sites_to_check.xlsx")

In [7]:
#sites_to_check_df
df

,site,url_to_import,esito,lingua
0,switchfan.org,switchfan.org,"[['switch', 35], ['nintendo', 30], ['mario', 1...",[fr:0.9999963316595247]
1,contioutra.com,contioutra.com,"[['conti', 15], ['outra', 13], ['filmes', 6], ...",[pt:0.999997725359873]
2,ricordinvaligia.it,ricordinvaligia.it,"[['della', 12], ['settembre', 8], ['agosto', 8...",[it:0.9999948012100419]
3,sportlegnano.it,sportlegnano.it,"[['serie', 36], ['legnano', 32], ['saronno', 2...",[it:0.9999965456651655]
4,geek-vintage.com,geek-vintage.com,"[['classé', 11], ['article', 6], ['vidéo', 5],...",[fr:0.9999967650119764]
...,...,...,...,...
65728,nexofin.com,NaN,vuoto,NaN
65729,nexoseguros.com.br,NaN,vuoto,NaN
65730,nextext.us,NaN,vuoto,NaN
65731,next-gamer.de,NaN,vuoto,NaN


In [ ]:
#spostare su file separato

def removeHtmlTags(lista):
    text = re.sub("\<(.*?)\>","",home_page_content)
    return text


def stripPunctuations(home_page_content):
    
    #bad_chars = "?.,;:!\'"
    bad_chars = ["\n","\t","?",".",",",";",":","!","\\","’"]
    for char in home_page_content:
        if char in bad_chars:
            home_page_content = home_page_content.replace(char,"")
        
    return home_page_content
        


def avoidStopWords(wordlist, stop_words):
    for word in wordlist:
        if word not in stop_words:
            print(word)
            list_saved_words.append(word)
    return list_saved_words


def removeWordsWithBadChars(content):
    bad_list = ["/","(",")", "&", "{","’"]
    bad_list += ["}", "h1", "h2", "h3",]
    bad_list += ["px","_","\\","&","@", "#","-","%","'"]
    bad_list += ["1","2","3","4","5","6","7","8","9","0"]
    for bad_char in bad_list:
        for word in content:
            if bad_char in word:
                content.remove(word)
    return content


def CountFrequency(list_saved_words): 
    lista_2 = []
    
    for item in list_saved_words:
        local_list = [item, list_saved_words.count(item)]
        lista_2.append(local_list)
    
    return lista_2


def removeDuplicates(list_saved_words):
    unique_keywords = []
    for item in list_saved_words:
        if item not in unique_keywords:
            unique_keywords.append(item)
    return unique_keywords



def sortList(sub_li): 
  
    # key is set to sort using second element of  
    # sublist lambda has been used 
    sub_li.sort(reverse=True, key = lambda x: x[1]) 
    return sub_li 


In [9]:
def elaborate_web_page(url, df):
    home_page_content = ""
    temp_keyword_frequencies = [] #serve per eliminare delle stop-words extra
    frequency_dict = {}
    all_words = [] #serve per eliminare delle stop-words extra, superato 
    print(url)
    
    if "http://" not in url: #togliendo l'http dal df e poi dal file excel rendo visibile che l'url è già stato controllato
        print("già fatto")
        return
    else:
        driver = webdriver.Chrome("webdriver/chromedriver")
        home_page_content_saved = []
        
        try:
            driver.get(url)
            print("url preso")
            time.sleep(5) #non sono sicuro che sia necessario, il mio timore è che certe pagine non carichino per intero (numero parole sempre diverso)
        except:
            print("non è un sito")
            df["esito"][index] = "error"
            return
            
        home_page_content = driver.find_element_by_tag_name('body').text
        home_page_content = str(home_page_content)
        print(home_page_content[:50])
        home_page_language = detect_langs(home_page_content)
        df["lingua"][url] = home_page_language
        print(home_page_language)
        print(df[:10])

        home_page_content = home_page_content.lower()
        home_page_content = removeHtmlTags(home_page_content)
        home_page_content = stripPunctuations(home_page_content)
        home_page_content = home_page_content.split(" ")
        #home_page_content diventa lista


        for word in home_page_content:
            if len(word)> 4 and len(word)< 20: 
                home_page_content_saved.append(word)
                home_page_content = home_page_content_saved

        #togliamo parole con bad_char
        home_page_content = removeWordsWithBadChars(home_page_content)
        #all_words += home_page_content

        #calcoliamo la frequenza, ma siamo sempre in una lista
        home_page_content = CountFrequency(home_page_content)

        #togliamo le keyword che sono state usate una volta sola
        temp_keyword_frequencies = []

        for item in home_page_content:
            if item[1] >= 2:
                temp_keyword_frequencies.append(item)
            home_page_content = temp_keyword_frequencies

        #sistemino per eliminare un po' di cripto-stopwords
        frequent_keywords = []

        for item in home_page_content:
            if item[1] >= 4:
                frequent_keywords.append(item)
        #print(frequent_keywords)


        #togliamo i duplicati
        home_page_content = removeDuplicates(home_page_content)
        #print(home_page_content)   

        #ordiniamo
        home_page_content = sortList(home_page_content)
        print("lunghezza finale: "+len(home_page_content))
        home_page_content = str(home_page_content) #prima di inserire i valori sul foglio excel trasformiamo in stringa

        df["esito"][url] = home_page_content
        df["url_to_import"][url] = df["url_to_import"][url].replace("http://","")
        driver.close()

    #finalmente si trasforma tutto in dizionario
    #il dizionario ha come key il sito, come value il mega-listone di keyword+frequency     
    #frequency_dict[url] = home_page_content



#si ricomincia da capo con un nuovo sito
    

    
#print(frequency_dict)

In [10]:
urls = df["url_to_import"]

for url in urls:
    elaborate_web_page(url, df)
    

switchfan.org
già fatto
contioutra.com
già fatto
ricordinvaligia.it
già fatto
sportlegnano.it
già fatto
geek-vintage.com
già fatto
http://fiorentinanews.com
url preso
Il rispetto della tua privacy è la nostra priorità
[it:0.9999957251535561]
                            site                         url_to_import  \
0                  switchfan.org                         switchfan.org   
1                 contioutra.com                        contioutra.com   
2             ricordinvaligia.it                    ricordinvaligia.it   
3                sportlegnano.it                       sportlegnano.it   
4               geek-vintage.com                      geek-vintage.com   
5             fiorentinanews.com             http://fiorentinanews.com   
6        fiorentinaneewfwews.com      http://fiorentinanewswfwfe4v.com   
7                 infocilento.it                 http://infocilento.it   
8  garotasquecurtemanimes.com.br  http://garotasquecurtemanimes.com.br   
9                b

NameError: name 'home_page_content' is not defined

In [14]:
print(df["lingua"][url])
print(df["site"][url])

[it:0.9999957251535561]


KeyError: 'http://fiorentinanews.com'

In [ ]:
df[:10]

In [ ]:
df.to_excel("sites_to_check.xlsx",index=False)

In [ ]:
#df.to_excel("sites_to_check_sbagliato.xlsx",index=False)

In [ ]:
#eliminare stopwords extra
from pandas import DataFrame
#df_all_words = DataFrame(frequent_keywords)
df_all_words
df_all_words.to_excel("frequent_keywords.xlsx")